In [1]:
import os
import sys
os.environ["SPARK_HOME"]="/home/user1/trainig_material/spark-2.4.4-bin-hadoop2.7"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["HIVE_HOME"] ="/usr/lib/hive"
#os.environ["PATH"] =os.environ["HIVE_HOME"]+"/bin"
#os.environ["JAVA_HOME"]="/usr/java/jdk1.7.0_67-cloudera"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
#os.environ["PYSPARK_PYTHON"] = "/home/cloudera/anaconda3/bin/python3.7" 
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/home/cloudera/anaconda3/bin/python3.7"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")


os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [2]:
# Spark
from pyspark import SparkContext, SparkConf
# Spark Streaming
from pyspark.streaming import StreamingContext
# Kafka
#from pyspark.streaming.kafka import KafkaUtils
# json parsing
from pyspark.sql import Row,SQLContext
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext,HiveContext

In [3]:
sc = SparkContext("local", "twitter")
sc.setLogLevel("WARN")

In [ ]:
sc.stop()

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [7]:
#spark.sql("show databases").show()

In [8]:
import pandas as pd
from datetime import datetime, timedelta
import json

import tweepy
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob

from kafka import KafkaClient
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka import TopicPartition
from kafka.errors import KafkaError

In [28]:
# not utilizing a structured streaming environment, this will allow us to save tweets in consumer 
#nd not neet to run producer and streaming simultaneosly avoiding error:420
consumer = KafkaConsumer('final_twitter', group_id=None, auto_offset_reset='earliest', bootstrap_servers=['localhost:9092'], value_deserializer=lambda m: json.loads(m)) 
#list of dicts, this will save tweets from consumer
rows = []

for msg in consumer:
    # this converts consumer to type dict
    row = msg.value
    # pull the hashtags
    if len(row['entities']['hashtags']) > 0:
        hashtag_text = row['entities']['hashtags'][0]['text']
    else:
        continue
        hashtag_text = row['entities']['hashtags']
    # create a row and add it to the list, we only need Tweet Id, User Id, Text, hashtags, and event time
    row = Row(TweetID=row['id'], Text=row['text'], UserID=row['user']['id'], Hashtags=hashtag_text, Event_time=row['created_at'])
    rows.append(row)
    print(row)

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
Tokyo2020
Row(Event_time='Sat Jul 24 20:22:57 +0000 2021', Hashtags='Tokyo2020', Text='I agree with @Devin_Heroux. I would be happy if Japanese 7-11 catered my wedding. \n#Tokyo2020 \n https://t.co/VYres1sv3U', TweetID=1419030363054088195, UserID=22720902)
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
Olympics2021
Row(Event_time='Sat Jul 24 20:22:58 +0000 2021', Hashtags='Olympics2021', Text='RT @_AfricanSoil: Black population in USA is less than 20% but check their #Olympics2021 team\n\nIn South Africa, 80% + of the population is…', TweetID=1419030365331673273, UserID=82641042)
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
Olympics
Row(Event_time='Sat Jul 24 20:22:58 +0000 2021', Hashtags='Olympics', Text='RT @mrdanwalker: Lots of people have asked about the full @mariadecezar #Olympics incident today 🏖😂\n5 years ago this week, on Copacaba

KeyboardInterrupt: 

In [22]:
# sanity check
print(rows)

[Row(Event_time='Sat Jul 24 20:22:57 +0000 2021', Hashtags='Tokyo2020', Text='I agree with @Devin_Heroux. I would be happy if Japanese 7-11 catered my wedding. \n#Tokyo2020 \n https://t.co/VYres1sv3U', TweetID=1419030363054088195, UserID=22720902), Row(Event_time='Sat Jul 24 20:22:58 +0000 2021', Hashtags='Olympics2021', Text='RT @_AfricanSoil: Black population in USA is less than 20% but check their #Olympics2021 team\n\nIn South Africa, 80% + of the population is…', TweetID=1419030365331673273, UserID=82641042), Row(Event_time='Sat Jul 24 20:22:58 +0000 2021', Hashtags='Olympics', Text='RT @mrdanwalker: Lots of people have asked about the full @mariadecezar #Olympics incident today 🏖😂\n5 years ago this week, on Copacabana be…', TweetID=1419030367957397510, UserID=360518857), Row(Event_time='Sat Jul 24 20:22:58 +0000 2021', Hashtags='bronze', Text='RT @Olympics: It’s #bronze for @DariaBilodid of #UKR in the #judo women’s 48 kg event. \n\nShe takes this medal on her #Olympics debut! \n

In [25]:
# convert the list of dicts to a spark dataframe as per the requirements
df = spark.createDataFrame(rows)

In [26]:
df.show()

+--------------------+--------------------+--------------------+-------------------+-------------------+
|          Event_time|            Hashtags|                Text|            TweetID|             UserID|
+--------------------+--------------------+--------------------+-------------------+-------------------+
|Sat Jul 24 20:22:...|           Tokyo2020|I agree with @Dev...|1419030363054088195|           22720902|
|Sat Jul 24 20:22:...|        Olympics2021|RT @_AfricanSoil:...|1419030365331673273|           82641042|
|Sat Jul 24 20:22:...|            Olympics|RT @mrdanwalker: ...|1419030367957397510|          360518857|
|Sat Jul 24 20:22:...|              bronze|RT @Olympics: It’...|1419030369039360002|         4594967060|
|Sat Jul 24 20:23:...|            Olympics|RT @EJinAction: T...|1419030374106214405|           26209691|
|Sat Jul 24 20:23:...|       TokyoOlympics|RT @mtgreenee: So...|1419030380645167105|1368422614725767169|
|Sat Jul 24 20:23:...|            Olympics|RT @todokugo